<a href="https://colab.research.google.com/github/maydogan23/kalbur/blob/master/Copy_of_spark0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

# Install java
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

# Install pyspark
! pip install --ignore-installed -q pyspark==2.4.4

# Install Spark NLP
! pip install --ignore-installed -q spark-nlp==2.4.5

openjdk version "1.8.0_242"
OpenJDK Runtime Environment (build 1.8.0_242-8u242-b08-0ubuntu3~18.04-b08)
OpenJDK 64-Bit Server VM (build 25.242-b08, mixed mode)
     |████████████████████████████████| 215.7MB 75kB/s 
     |████████████████████████████████| 204kB 53.1MB/s 
     |████████████████████████████████| 112kB 34.6MB/s 


In [0]:
import os
import sys

from pyspark.sql import SparkSession
from pyspark.ml import Pipeline

from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *

import pandas as pd

import sparknlp

spark = sparknlp.start(gpu=True) # for GPU training >> sparknlp.start(gpu=True)

from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
import pandas as pd

print("Spark NLP version", sparknlp.version())

print("Apache Spark version:", spark.version)

spark

Spark NLP version 2.4.5
Apache Spark version: 2.4.4


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
trainDataset = spark.read \
      .option("header", True) \
      .csv("drive/My Drive/train.csv")
trainDataset=trainDataset.drop("title")

trainDataset.show(truncate=50)

+--------+--------------------------------------------------+
|category|                                       description|
+--------+--------------------------------------------------+
|       3|Reuters - Short-sellers, Wall Street's dwindlin...|
|       3|Reuters - Private investment firm Carlyle Group...|
|       3|Reuters - Soaring crude prices plus worries\abo...|
|       3|Reuters - Authorities have halted oil export\fl...|
|       3|AFP - Tearaway world oil prices, toppling recor...|
|       3|Reuters - Stocks ended slightly higher on Frida...|
|       3|AP - Assets of the nation's retail money market...|
|       3|USATODAY.com - Retail sales bounced back a bit ...|
|       3|"Forbes.com - After earning a PH.D. in Sociolog...|
|       3| NEW YORK (Reuters) - Short-sellers, Wall Stree...|
|       3| NEW YORK (Reuters) - Soaring crude prices plus...|
|       3| TEHRAN (Reuters) - OPEC can do nothing to dous...|
|       3| JAKARTA (Reuters) - Non-OPEC oil exporters sho...|
|       

In [0]:
trainDataset.count()

from pyspark.sql.functions import col

trainDataset.groupBy("category") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------+-----+
|category|count|
+--------+-----+
|       1|30000|
|       3|30000|
|       4|30000|
|       2|30000|
+--------+-----+



In [0]:

testDataset = spark.read \
      .option("header", True) \
      .csv("drive/My Drive/test.csv")
testDataset=testDataset.drop("title")

testDataset.show(truncate=50)

+--------+--------------------------------------------------+
|category|                                       description|
+--------+--------------------------------------------------+
|       3|Unions representing workers at Turner   Newall ...|
|       4|SPACE.com - TORONTO, Canada -- A second\team of...|
|       4|AP - A company founded by a chemistry researche...|
|       4|AP - It's barely dawn when Mike Fitzpatrick sta...|
|       4|AP - Southern California's smog-fighting agency...|
|       4|"The British Department for Education and Skill...|
|       4|                                  "\""Sven Jaschan|
|       4|\FOAF/LOAF  and bloom filters have a lot of int...|
|       4|"Wiltshire Police warns about ""phishing"" afte...|
|       4|In its first two years, the UK's dedicated card...|
|       4| LOS ANGELES (Reuters) - A group of technology ...|
|       4| LOS ANGELES (Reuters) - Apple Computer Inc.&lt...|
|       4| AMSTERDAM (Reuters) - Free Record Shop, a Dutc...|
|       

In [0]:
document_assembler = DocumentAssembler() \
    .setInputCol("description") \
    .setOutputCol("document")
    
tokenizer = Tokenizer() \
  .setInputCols(["document"]) \
  .setOutputCol("token")
  

spell_checker = NorvigSweetingApproach() \
    .setInputCols(["token"]) \
    .setOutputCol("checked") \
    .setDictionary("drive/My Drive/coca2017.txt", "[a-zA-Z]+")
    
normalizer = Normalizer() \
    .setInputCols(["checked"]) \
    .setOutputCol("normalized")

stopwords_cleaner = StopWordsCleaner()\
      .setInputCols("normalized")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)

lemma = LemmatizerModel.pretrained('lemma_antbnc') \
    .setInputCols(["cleanTokens"]) \
    .setOutputCol("lemma")

word_embeddings = BertEmbeddings.pretrained('bert_base_cased', 'en')\
  .setInputCols(["document", "lemma"])\
  .setOutputCol("embeddings")

embeddingsSentence = SentenceEmbeddings() \
      .setInputCols(["document", "embeddings"]) \
      .setOutputCol("sentence_embeddings") \
      .setPoolingStrategy("AVERAGE")
    


classsifierdl = ClassifierDLApproach()\
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("class")\
  .setLabelColumn("category")\
   .setBatchSize(64)\
      .setMaxEpochs(20)\
      .setLr(0.01)\
      .setDropout(0.2)\
      .setEnableOutputLogs(True)

clf_pipeline = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            spell_checker,
            normalizer,
            stopwords_cleaner, 
            lemma,
            word_embeddings,
            embeddingsSentence,
                 
            classsifierdl])

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]
bert_base_cased download started this may take some time.
Approximate size to download 389.2 MB
[OK!]


In [0]:
%%time
clf_pipelineModel = clf_pipeline.fit(trainDataset)

In [0]:
!cd ~/annotator_logs && ls -l

In [0]:
!cat ~/annotator_logs/ClassifierDLApproach_824221680502.log

In [0]:
preds = clf_pipelineModel.transform(testDataset)

preds.select('category','description',"class.result").show(10, truncate=80)

In [0]:
preds_df = preds.select('category','description',"class.result").toPandas()

# The result is an array since in Spark NLP you can have multiple sentences.
# Let's explode the array and get the item(s) inside of result column out
preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

In [0]:
# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report

print (classification_report(preds_df['result'], preds_df['category']))
